In [ ]:
import requests
import random
import time
import socket
import http.client
import json
import pandas as pd

In [ ]:
def get_html(index=0, minP=None, maxP=None, minBed=None, maxBed=None):
    base_url = 'https://www.rightmove.co.uk/api/_search?'
    payload = {
        "index": index*24,
        "sortType": 6, #大于1000条可通过排序正反抓取两次
        "numberOfPropertiesPerPage": 24,
        "locationIdentifier": "REGION^79192",
        "viewType": "LIST",
        "includeLetAgreed": "true",
        "minBedrooms": minBed,
        "maxBedrooms": maxBed,
        "minPrice": minP,
        "maxPrice": maxP,
        "propertyTypes": ["flat", "detached", "semi-detached", "terraced", "bungalow"],
        "must_have": [],
        "dontShow": ["student", "retirement"],
        "channel": "RENT",
        "areaSizeUnit":"sqft",
        "currencyCode":"GBP",
        "isFetching":"false"
        }
    header = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en,en-US;q=0.8,zh-CN;q=0.7,zh;q=0.5,zh-TW;q=0.3,zh-HK;q=0.2',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0'
            }
    timeout = random.choice(range(80, 100))
    while True:
        try:
            response = requests.get(base_url, params=payload, headers = header, timeout = timeout)
            response.encoding = 'utf-8'
            break
        except socket.timeout as e:
            print(e)
            time.sleep(random.choice(range(20, 60)))
        except socket.error as e:
            print(e)
            time.sleep(random.choice(range(0, 60)))
        except http.client.BadStatusLine as e:
            print(e)
            time.sleep(random.choice(range(30, 60)))
        except http.client.IncompleteRead as e:
            print(e)
            time.sleep(random.choice(range(20, 60)))
    print(response.url)
    return response.json()

def data_output(data, filename):
    with open(filename, 'a', errors='ignore', newline='', encoding='utf-8') as f:
        f_csv = csv.writer(f)
        f_csv.writerows(data)
    
if __name__ == '__main__':
    pro_df = pd.DataFrame()
    mylist1 = [None, 500, 600]
    mylist2 = [600, 700, 800, 900, 1000]
    mylist3 = [1000, 1100, 1250, None]
    for j in range(len(mylist2)-1):
        html = get_html(index=0, minP=mylist2[j], maxP=mylist2[j+1])
        page = html['pagination']['total']
        for i in range(page):
            html = get_html(index=i, minP=mylist2[j], maxP=mylist2[j+1])
            pro_df = pro_df.append(pd.DataFrame(html['properties']))
            time.sleep(random.choice(range(2)))

  #  data_output(result, 'rightmove.csv')

In [ ]:
    for j in range(len(mylist3)-1): #mylist1 and mylist3
        html = get_html(index=0, minP=mylist3[j], maxP=mylist3[j+1])
        page = html['pagination']['total']
        for i in range(page):
            html = get_html(index=i, minP=mylist3[j], maxP=mylist3[j+1])
            pro_df = pro_df.append(pd.DataFrame(html['properties']))
            time.sleep(random.choice(range(2)))

In [ ]:
pro_df.columns.tolist()

##### 重置行索引

In [ ]:
#pro_df = pro_df.reset_index()
pro_df

##### 查找重复

In [ ]:
dups = pro_df.pivot_table(index=['id'], aggfunc='size')
#dups = pd.DataFrame(dups)
dups[dups==2]

In [ ]:
pro_df.index[pro_df['id']==50810189,].tolist()

In [ ]:
pro_df.loc[pro_df['identifier'].isin(dups.index[:])]

##### 去重

In [ ]:
pro_df_dedup = pro_df.drop_duplicates("id")
pro_df_dedup

In [ ]:
pro_df_dedup.to_csv('rightmove1.csv')